<a href="https://colab.research.google.com/github/Hritika-H/Book-recommendation-System/blob/main/BookRecSys1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


##Book Recommendation System
---

This Book Recommendation System is a cross domain recommendation system. The User will pick a Movie and get Book recommendations. In this content based recommendation system, we use two different datasets: 

*   for movies(https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset)
*   for books (https://www.kaggle.com/yehyachali/top2k-books-with-descriptions)


###Mount Google Drive

In [ ]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#changing current dir
%cd '/content/drive/MyDrive/MSc_Project'

/content/drive/MyDrive/MSc_Project


In [ ]:
#unzipping
#!unzip -uq /content/drive/MyDrive/MSc_Project/movies_metadata.csv.zip -d /content/drive/MyDrive/MSc_Project
#!unzip -uq /content/drive/MyDrive/MSc_Project/keywords.csv.zip -d /content/drive/MyDrive/MSc_Project
#!unzip -uq /content/drive/MyDrive/MSc_Project/top2k_book_descriptions.csv.zip -d /content/drive/MyDrive/MSc_Project

###Data Preprocessing For The Movie Dataset

In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
%matplotlib inline

In [ ]:
movies = pd.read_csv("/content/drive/MyDrive/MSc_Project/movies_metadata.csv")
print(movies.columns)
movies.head()

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
movies['genres'][0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

The genres in Movies dataset are in a dictionary format however the type is string. Use literal_eval function to get the dictionary, then all we need is to select names of the genres.

In [ ]:
movies['genres'] = movies['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [ ]:
movies['genres'][0]

['Animation', 'Comedy', 'Family']

We need to get the keywords to add to our data.

In [ ]:
keywords = pd.read_csv('/content/drive/MyDrive/MSc_Project/keywords.csv')

In [ ]:
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [ ]:
def clean_ids(x):
    try:
        return int(x)
    except:
        return np.nan

movies['id'] = movies['id'].apply(clean_ids)
movies = movies[movies['id'].notnull()]

Joining movies and keywords dataframes

In [ ]:
movies['id'] = movies['id'].astype('int')
keywords['id'] = keywords['id'].astype('int')

movies = movies.merge(keywords, on='id')

movies.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [ ]:
movies["keywords"][0]

"[{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': 'toy'}, {'id': 5202, 'name': 'boy'}, {'id': 6054, 'name': 'friendship'}, {'id': 9713, 'name': 'friends'}, {'id': 9823, 'name': 'rivalry'}, {'id': 165503, 'name': 'boy next door'}, {'id': 170722, 'name': 'new toy'}, {'id': 187065, 'name': 'toy comes to life'}]"

Same as the genres, we need to convert them from string to dictionary.

In [ ]:
movies["keywords"] = movies["keywords"].apply(literal_eval)

In [ ]:
movies["keywords"][0]

[{'id': 931, 'name': 'jealousy'},
 {'id': 4290, 'name': 'toy'},
 {'id': 5202, 'name': 'boy'},
 {'id': 6054, 'name': 'friendship'},
 {'id': 9713, 'name': 'friends'},
 {'id': 9823, 'name': 'rivalry'},
 {'id': 165503, 'name': 'boy next door'},
 {'id': 170722, 'name': 'new toy'},
 {'id': 187065, 'name': 'toy comes to life'}]

generate_list function will help you to select as many keywords you need.

In [ ]:
def generate_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        if len(names) > 10:
            names = names[:10]
        return names
    return []

movies['keywords'] = movies['keywords'].apply(generate_list)
movies['genres'] = movies['genres'].apply(lambda x: x[:10])

movies[['title', 'keywords', 'genres']].head()

,title,keywords,genres
0,Toy Story,"[jealousy, toy, boy, friendship, friends, riva...","[Animation, Comedy, Family]"
1,Jumanji,"[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]"
2,Grumpier Old Men,"[fishing, best friend, duringcreditsstinger, o...","[Romance, Comedy]"
3,Waiting to Exhale,"[based on novel, interracial relationship, sin...","[Comedy, Drama, Romance]"
4,Father of the Bride Part II,"[baby, midlife crisis, confidence, aging, daug...",[Comedy]


Now we have to change them back to string tokens so that we can add them all to our data

In [ ]:
def sanitize(x):
    if isinstance(x, list):
        return [str.lower(i.replace(' ','')) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(' ', ''))
        else:
            return ''

for feature in ['genres', 'keywords']:
    movies[feature] = movies[feature].apply(sanitize)

Our movies dataframe includes the name of the movie, genres, overview, and the keywords(or sub-genres)

In [ ]:
def movie_soup(x):
    return  x["title"] + " " + " ".join(x['genres']) + " "+x['overview']+" "+" ".join(x['keywords'])

movies['overview'] = movies['overview'].fillna('')
movies['title'] = movies['title'].fillna('')
movies['soup'] = movies.apply(movie_soup, axis=1)

In [ ]:
movies.loc[movies['title']=="The Matrix",'soup'].values

array(['The Matrix action sciencefiction Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth. savingtheworld artificialintelligence manvsmachine philosophy prophecy martialarts selfsacrifice fight insurgence virtualreality'],
      dtype=object)

###Data Preprocessing For The Books Dataset

In [ ]:
books = pd.read_csv("/content/drive/MyDrive/MSc_Project/top2k_book_descriptions.csv", index_col=0)
print(books.columns)
books.head()

Index(['id', 'book_id', 'best_book_id', 'work_id', 'books_count', 'isbn',
       'isbn13', 'authors', 'original_publication_year', 'original_title',
       'title', 'language_code', 'average_rating', 'ratings_count',
       'work_ratings_count', 'work_text_reviews_count', 'ratings_1',
       'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 'image_url',
       'small_image_url', 'goodreads_book_id', 'tag_name', 'description'],
      dtype='object')


,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,goodreads_book_id,tag_name,description
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,2767052,[],WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,3,"['books', 'bbc', 'books', 'every', 'man', 'sho...",Harry Potter's life is miserable. His parents ...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,41865,[],About three things I was absolutely positive.F...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,2657,"['mercenaries', 'mercenary', 'mercer', 'mayer'...",The unforgettable novel of a childhood in a sl...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,4671,[],Alternate Cover Edition ISBN: 0743273567 (ISBN...


In [ ]:
books['tag_name'][1]

"['books', 'bbc', 'books', 'every', 'man', 'should', 'read', 'books', 'own', 'for', 'books', 'that', 'define', 'the', 'books', 'to', 'read', 'books', 'to', 'read', 'before', 'you', 'die', 'books', 'to', 'read', 'in', 'lifetime', 'books', 'to', 'read', 'my', 'version', 'bullets', 'children', 'books', 'essential', 'novels', 'essential', 'novels', 'poster', 'fiction', 'greatest', 'greatest', 'books', 'ever', 'written', 'greatest', 'novels', 'in', 'list', 'modern', 'library', 'must', 'read', 'books', 'must', 'read', 'books', 'the', 'essential', 'novels', 'everyone', 'should', 'read', 'recommended', 'year', 'war', 'years', 'war', 'book', 'before', 'kindergarten', 'book', 'goal', 'books', 'books', 'before', 'kindergarten', 'books', 'to', 'read', 'before', 'you', 'die', 'by', 'good', 'books', 'harry', 'novels', 'plus', 'pages', 'before', 'you', 'die', 'book', 'book', 'list', 'books', 'books', 'books', 'before', 'die', 'books', 'before', 'you', 'die', 'books', 'challenge', 'books', 'have', 're

In [ ]:
books['tag_name'] = books['tag_name'].apply(lambda x: literal_eval(x) if literal_eval(x) else np.nan)
books = books[books['description'].notnull() | books['tag_name'].notnull()]
books = books.fillna('')

In [ ]:
def book_soup(x):
    soup = x["original_title"]+" "+x["description"]+" "+" ".join(x['tag_name'])+" "+x["authors"]
    return soup

In [ ]:
books["soup"] = books.apply(book_soup, axis=1)

In [ ]:
soups = pd.concat([movies['soup'],books['soup']],ignore_index=True)

In [ ]:
count = CountVectorizer(stop_words = "english")
count.fit(soups)

movies_matrix = count.transform(movies['soup'])
books_matrix = count.transform(books['soup'])

books_matrix.shape, movies_matrix.shape

((1987, 98296), (46482, 98296))

In [ ]:
cosine_sim = cosine_similarity(movies_matrix, books_matrix)

In [ ]:
movies = movies.reset_index()
indices = pd.Series(movies.index, index=movies['title'].apply(lambda x: x.lower() if x is not np.nan else "")).drop_duplicates()

In [ ]:
def content_recommender(title):
    idx = indices[title.lower()]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)
    
    sim_scores = sim_scores[:10]

    book_indices = [i[0] for i in sim_scores]

    return books.iloc[book_indices]

In [ ]:
print(content_recommender("I, Robot"))

        id   book_id  best_book_id   work_id  books_count        isbn  \
431    432     41804         41804   1796026          218   553803700   
1789  1914     41811         41811    140376          139   586008357   
252    253  16160797      16160797  22002305           48   316206849   
943    944     10584         10584     14015          132   451188462   
0        1   2767052       2767052   2792775          272   439023483   
714    715     12898         12898   2722054          139   435233076   
1794  1924     34517         34517   1796454           80   552152951   
832    833      5355          5355   1254155          112   385339666   
784    785     32669         32669   5054704           91   425144372   
584    585     13615         13615   1782155           44  1421501686   

               isbn13                         authors  \
431   9780553803710.0                    Isaac Asimov   
1789  9780586008360.0                    Isaac Asimov   
252   9780316206850.0  Ro

In [ ]:
print(content_recommender("Toy Story"))

        id   book_id  best_book_id   work_id  books_count        isbn  \
495    496      1295          1295   1584694          161   553381679   
1120  1125   6969361       6969361   7208552           21  8129115301   
403    404  11367726      11367726  16298550           76   385344228   
1068  1072   7114761       7114761  10424899           48   312554168   
802    803    271199        271199    542633           17  1591826039   
943    944     10584         10584     14015          132   451188462   
512    513    561909        561909    878114           83   553256696   
237    238    242006        242006    828773           79  031242227X   
10      11     77203         77203   3295919          283  1594480001   
1949  2450  16151178      16151178  21547521           31               

               isbn13                                            authors  \
495   9780553381670.0                                       Jean M. Auel   
1120  9788129115300.0                       

In [ ]:
!pip3 install -q ipywidgets
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/usr/etc/jupyter/nbconfig/notebook.json
      - Validating: OK
Paths used for configuration of notebook: 
    	/usr/etc/jupyter/nbconfig/notebook.json


In [ ]:
import ipywidgets
from IPython.display import HTML
def showhtml(recommendations):
    html = ' '.join([f"""
     <div class="flip-card">
      <div class="flip-card-inner">
        <div class="flip-card-front">
          <img src="{recommendations.iloc[i]['image_url']}" alt="Avatar" style="width:300px;height:300px;">
        </div>
        <div class="flip-card-back">
          <h4>{recommendations.iloc[i]['title']}</h4>
          <p>by {recommendations.iloc[i]['authors']}</p>
        </div>
      </div>
    </div> """ for i in range(10)])
    html = "<div class='grid'>"+html+"</div>"
    html +="""<style>
    .flip-card {
      background-color: transparent;
      width: 200px;
      height: 300px;
      border: 1px solid #f1f1f1;
    }

    .flip-card-inner {
      position: relative;
      width: 100%;
      height: 100%;
      text-align: center;
      transition: transform 0.8s;
      transform-style: preserve-3d;
    }

    .flip-card:hover .flip-card-inner {
      transform: rotateY(180deg);
    }

    .flip-card-front, .flip-card-back {
      position: absolute;
      width: 100%;
      height: 100%;
      -webkit-backface-visibility: hidden; /* Safari */
      backface-visibility: hidden;
    }

    .flip-card-front {
      background-color: #bbb;
      color: black;
    }

    .flip-card-back {
    padding:10px;
      background-color: dodgerblue;
      color: white;
      transform: rotateY(180deg);
    }
    .grid {
        display: grid;
        grid-template-columns: 30% 30% 30%;
        grid-template-rows: 25% 25% 25%;
        grid-gap: 5%;
    }
    </style>"""
    return html


def show_books(movie_name='I, Robot'):
    recommendations = content_recommender(movie_name)
#     for i in range(10):
#         disPic(recommendations["image_url"].iloc[i])
#         print(recommendations["original_title"].iloc[i])
#         print(recommendations["description"].iloc[i])
    display(HTML(showhtml(recommendations)))
display(ipywidgets.interact(show_books))

interactive(children=(Text(value='I, Robot', description='movie_name'), Output()), _dom_classes=('widget-inter…

<function __main__.show_books>

In [ ]:
show_books(movie_name='Pocahontas') #Pocahontas Free Willy